In [1]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 2.8 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=9cdd571f83a9d4b2920e253a452820e27a3b29bc3e817f01126baccc19cef0bf
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [2]:
import numpy as np  # Importing NumPy for numerical operations
import pandas as pd  # Importing Pandas for data manipulation
import matplotlib.pyplot as plt  # Importing Matplotlib for plotting
from lifelines import KaplanMeierFitter  # Importing Kaplan-Meier estimator from lifelines
from lifelines.statistics import logrank_test  # Importing log-rank test for survival comparison


In [3]:
from google.colab import files
uploaded= files.upload()

Saving GBM_patient_data.csv to GBM_patient_data.csv


In [5]:
pd_data= pd.read_csv('/content/GBM_patient_data.csv')
pd_data.head()

,age (years),"sex (0=Female, 1=Male)","MGMT_promotor_methylated (0=WT, 1=Methylated)","IDH_mutant (0=WT, 1=Mutant)","treatment (0=TMZ-untreated, 1=TMZ-treated alone, 2=Mi1+TMZ)","event (1=event occurred, 0=censored)",survival_time (days)
0,64.967142,0,0,1,0,1,194.181664
1,58.617357,0,0,0,0,1,73.346847
2,66.476885,1,0,0,1,1,360.114365
3,75.230299,1,0,1,1,1,219.539143
4,57.658466,0,1,0,0,0,516.734029


In [15]:
pd_data_0=pd_data[pd_data['treatment (0=TMZ-untreated, 1=TMZ-treated alone, 2=Mi1+TMZ)']==0]
pd_data_0.head()

,age (years),"sex (0=Female, 1=Male)","MGMT_promotor_methylated (0=WT, 1=Methylated)","IDH_mutant (0=WT, 1=Mutant)","treatment (0=TMZ-untreated, 1=TMZ-treated alone, 2=Mi1+TMZ)","event (1=event occurred, 0=censored)",survival_time (days)
0,64.967142,0,0,1,0,1,194.181664
1,58.617357,0,0,0,0,1,73.346847
4,57.658466,0,1,0,0,0,516.734029
9,65.425600,1,0,0,0,1,166.110538
14,42.750822,0,0,1,0,1,309.066622


In [16]:
pd_data_1=pd_data[pd_data['treatment (0=TMZ-untreated, 1=TMZ-treated alone, 2=Mi1+TMZ)']==1]
pd_data_1.head()

,age (years),"sex (0=Female, 1=Male)","MGMT_promotor_methylated (0=WT, 1=Methylated)","IDH_mutant (0=WT, 1=Mutant)","treatment (0=TMZ-untreated, 1=TMZ-treated alone, 2=Mi1+TMZ)","event (1=event occurred, 0=censored)",survival_time (days)
2,66.476885,1,0,0,1,1,360.114365
3,75.230299,1,0,1,1,1,219.539143
7,67.674347,1,0,0,1,0,379.813096
10,55.365823,1,0,0,1,1,272.075528
22,60.675282,0,0,1,1,1,491.067774


# **Check Normality for age between groups**

In [49]:
# apply shapiro-wilk test to age in each group
from scipy.stats import shapiro
stat, p= shapiro(pd_data_0['age (years)'])
print('stat=%.3f, p=%.3f' % (stat, p))

stat=0.994, p=0.245


In [50]:
stat, p= shapiro(pd_data_1['age (years)'])
print('stat=%.3f, p=%.3f' % (stat, p))

stat=0.994, p=0.294


# **checking equality of variance with Levene's test**

In [51]:
# Levene's test for equality of variance check
from scipy.stats import levene
stat, p= levene(pd_data_0['age (years)'], pd_data_1['age (years)'])
print('stat=%.3f, p=%.3f' % (stat, p))

stat=0.196, p=0.659


# **Two-sampled t-test**

In [52]:
from scipy.stats import ttest_ind
t_stat,p_val= ttest_ind(pd_data_0['age (years)'], pd_data_1['age (years)'])
print('t_stat=%.3f, p_val=%.3f' % (t_stat, p_val))

t_stat=-0.044, p_val=0.965


# **H0: the proportion of female/male is the same in both groups**

In [23]:
females_0=np.sum(pd_data_0['sex (0=Female, 1=Male)']==0)
males_0=sum(pd_data_0['sex (0=Female, 1=Male)']==1)
print('fem_0:', females_0)
print('male_0:', males_0)



fem_0: 139
male_0: 162


In [22]:
len(pd_data_0)

301

In [24]:
females_1=np.sum(pd_data_1['sex (0=Female, 1=Male)']==0)
males_1=sum(pd_data_1['sex (0=Female, 1=Male)']==1)
print('fem_1:', females_1)
print('male_1:', males_1)

fem_1: 165
male_1: 152


In [25]:
# make contigency table for group 0 and 1 with genders
contingency_table=np.array([[females_0, males_0], [females_1, males_1]])
print(contingency_table)

[[139 162]
 [165 152]]


In [26]:
# add column and row names to contigency table
contingency_table=pd.DataFrame(contingency_table, columns=['Female', 'Male'], index=['Group 0', 'Group 1'])
print(contingency_table)

         Female  Male
Group 0     139   162
Group 1     165   152


In [28]:
# make the genders rows instead of columns
contingency_table=contingency_table.transpose()
print(contingency_table)

        Group 0  Group 1
Female      139      165
Male        162      152


In [29]:
#do a chi-square test to the table for gender variables
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(contingency_table)
print('Chi-square statistic:', chi2)
print('p-value:', p)

Chi-square statistic: 1.9009175206818725
p-value: 0.16797565523431188


***p-value: 0.16797565523431188 >> 0.05 suggests that we fail to reject null hypothesis --> the proportion of males vs. females is roughly the same in both groups***

H0:

In [32]:
unmyl_0=sum(pd_data_0['MGMT_promotor_methylated (0=WT, 1=Methylated)']==0)
myl_0=sum(pd_data_0['MGMT_promotor_methylated (0=WT, 1=Methylated)']==1)
print(unmyl_0)
print(myl_0)

179
122


In [33]:
unmyl_1=sum(pd_data_1['MGMT_promotor_methylated (0=WT, 1=Methylated)']==0)
myl_1=sum(pd_data_1['MGMT_promotor_methylated (0=WT, 1=Methylated)']==1)
print(unmyl_1)
print(myl_1)

182
135


In [34]:
table=np.array([[unmyl_0,myl_0],[unmyl_1,myl_1]])
print(table)

[[179 122]
 [182 135]]


In [36]:
table=pd.DataFrame(table, columns=['WT', 'Methylated'], index=['Group 0', 'Group 1'])
table=table.transpose()
print(table)

            Group 0  Group 1
WT              179      182
Methylated      122      135


In [37]:
#do a chi-square test to the table for the WT and Methylated variables
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(table)
print('Chi-square statistic:', chi2)
print('p-value:', p)

Chi-square statistic: 0.1905208357777805
p-value: 0.6624834835489024


***p-value: 0.6624834835489024 >> 0.05 suggests that we fail to reject null hypothesis --> the proportion of WT vs. Methylated is roughly the same in both groups***

# **H0: The two groups don't have imbalance in distribution of IDH Mutation**

In [42]:
mutant_0=sum(pd_data_0['IDH_mutant (0=WT, 1=Mutant)']==0)
unmutant_0=sum(pd_data_1['IDH_mutant (0=WT, 1=Mutant)']==1)
print(mutant_0)
print(unmutant_0)


181
136


In [43]:
mutant_1=sum(pd_data_1['IDH_mutant (0=WT, 1=Mutant)']==1)
unmutant_1=sum(pd_data_1['IDH_mutant (0=WT, 1=Mutant)']==0)
print(unmutant_1)
print(mutant_1)



181
136


In [45]:
con_table=np.array([[mutant_0, unmutant_0],[mutant_1, unmutant_1]])
print(con_table)

[[181 136]
 [136 181]]


In [47]:
con_table= pd.DataFrame(con_table,columns=['Mutant', 'WT'], index=['Group 0', 'Group 1'])
con_table=con_table.transpose()
print(con_table)

        Group 0  Group 1
Mutant      181      136
WT          136      181


In [48]:
##do a chi-square test to the table for the  variables
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency(con_table)
print('Chi-square statistic:', chi2)
print('p-value:', p)

Chi-square statistic: 12.214511041009464
p-value: 0.00047419237021035945


p-value: 0.00047419237021035945 << 0.05 indicates that there's imbalance between two groups interms of IDH mutation.Therefore, we fail to reject  H0

In [12]:
len(pd_data)

1000